# Cyclistic Data Exploration using SQL

This notebook explores the Cyclistic bike trip data using SQL queries via SQLite CLI commands.

In [10]:
# Setup: Import CSV into SQLite database
# SQLite is built-in on most systems - no installation needed!

import subprocess
import os

csv_path = r"C:\Users\imonl\OneDrive\Počítač\UCD\3rd year\2nd Trimester\COMP30770 - Programming for Big Data\cyclistic_tripdata_2020.csv"
db_path = "cyclistic.db"

# Check if SQLite is available
result = subprocess.run(['sqlite3', '--version'], capture_output=True, text=True)
if result.returncode == 0:
    print(f"SQLite version: {result.stdout.strip()}")
else:
    print("SQLite not found. On Windows, you may need to install it or use Python's sqlite3 module.")
    print("Alternatively, SQLite comes pre-installed on Linux/Mac.")

FileNotFoundError: [WinError 2] The system cannot find the file specified

## 1. Import CSV into SQLite Database

First, let's import the CSV file into SQLite. We'll use Python to do this, then use SQLite CLI for queries.

In [11]:
# Import CSV into SQLite using Python (one-time setup)
import sqlite3
import pandas as pd

# Read CSV in chunks to handle large file
chunk_size = 100000
db_path = "cyclistic.db"

# Remove existing database if it exists
if os.path.exists(db_path):
    os.remove(db_path)

conn = sqlite3.connect(db_path)
print("Importing CSV into SQLite database...")

# Read and insert in chunks
for i, chunk in enumerate(pd.read_csv(csv_path, chunksize=chunk_size, low_memory=False)):
    chunk.to_sql('trips', conn, if_exists='append', index=False)
    if (i + 1) % 10 == 0:
        print(f"Imported {(i + 1) * chunk_size:,} rows...")

conn.close()
print(f"\nImport complete! Database saved to {db_path}")
print("Now you can use SQLite CLI commands below.")

Importing CSV into SQLite database...
Imported 1,000,000 rows...
Imported 2,000,000 rows...
Imported 3,000,000 rows...

Import complete! Database saved to cyclistic.db
Now you can use SQLite CLI commands below.


In [ ]:
## 2. Basic SQL Queries using SQLite CLI

#Now we can use bash commands with SQLite CLI!

### Count Total Rows

In [ ]:
# Using bash command with SQLite CLI
!sqlite3 cyclistic.db "SELECT COUNT(*) as total_rows FROM trips;"

### View Sample Data

In [ ]:
# View first 10 rows with headers
!sqlite3 -header -column cyclistic.db "SELECT * FROM trips LIMIT 10;"

### Table Schema

In [ ]:
# Get table schema
!sqlite3 -header -column cyclistic.db "PRAGMA table_info(trips);"

### Member vs Casual Riders

In [ ]:
# Count by member type
!sqlite3 -header -column cyclistic.db "SELECT member_casual, COUNT(*) as ride_count, ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM trips), 2) as percentage FROM trips GROUP BY member_casual ORDER BY ride_count DESC;"

### Bike Types

In [ ]:
# Count by bike type
!sqlite3 -header -column cyclistic.db "SELECT rideable_type, COUNT(*) as ride_count FROM trips GROUP BY rideable_type ORDER BY ride_count DESC;"

In [ ]:
### Top Start Stations

### Top End Stations

In [ ]:
# Top 10 start stations
!sqlite3 -header -column cyclistic.db "SELECT start_station_name, COUNT(*) as ride_count FROM trips WHERE start_station_name IS NOT NULL AND start_station_name != '' GROUP BY start_station_name ORDER BY ride_count DESC LIMIT 10;"

### NULL Value Check

In [ ]:
# Top 10 end stations
!sqlite3 -header -column cyclistic.db "SELECT end_station_name, COUNT(*) as ride_count FROM trips WHERE end_station_name IS NOT NULL AND end_station_name != '' GROUP BY end_station_name ORDER BY ride_count DESC LIMIT 10;"

In [ ]:
# Check for NULL values
!sqlite3 -header -column cyclistic.db "SELECT COUNT(*) - COUNT(ride_id) as null_ride_id, COUNT(*) - COUNT(started_at) as null_started_at, COUNT(*) - COUNT(ended_at) as null_ended_at, COUNT(*) - COUNT(member_casual) as null_member_casual FROM trips;"

### Rides by Day of Week

In [ ]:
# Rides by day of week
!sqlite3 -header -column cyclistic.db "SELECT strftime('%w', started_at) as day_of_week, CASE strftime('%w', started_at) WHEN '0' THEN 'Sunday' WHEN '1' THEN 'Monday' WHEN '2' THEN 'Tuesday' WHEN '3' THEN 'Wednesday' WHEN '4' THEN 'Thursday' WHEN '5' THEN 'Friday' WHEN '6' THEN 'Saturday' END as day_name, COUNT(*) as ride_count FROM trips WHERE started_at IS NOT NULL GROUP BY day_of_week ORDER BY day_of_week;"

In [ ]:
### Rides by Hour of Day

## Alternative: Using Bash SQL Commands

Below are examples using bash commands with DuckDB CLI (if installed) or SQLite.

In [ ]:
# Example: Using bash to run SQL queries with DuckDB CLI
# Note: This requires DuckDB CLI to be installed separately
# You can install it from: https://duckdb.org/docs/installation/

import subprocess
import os

csv_path = r"C:\Users\imonl\OneDrive\Počítač\UCD\3rd year\2nd Trimester\COMP30770 - Programming for Big Data\cyclistic_tripdata_2020.csv"

# Example SQL query as a string
sql_query = f"""
SELECT COUNT(*) as total_rows 
FROM read_csv_auto('{csv_path.replace(chr(92), "/")}');
"""

# Save SQL to a temporary file
with open('temp_query.sql', 'w') as f:
    f.write(sql_query)

print("SQL query saved to temp_query.sql")
print("\nTo run with DuckDB CLI (if installed), use:")
print(f"duckdb -c \"{sql_query.strip()}\"")
print("\nOr:")
print("duckdb < temp_query.sql")

### Bash SQL Commands (Run these in terminal)

Here are SQL commands you can run directly in bash/terminal:

**Using DuckDB CLI:**

```bash
# Count total rows
duckdb -c "SELECT COUNT(*) FROM read_csv_auto('cyclistic_tripdata_2020.csv');"

# Sample data
duckdb -c "SELECT * FROM read_csv_auto('cyclistic_tripdata_2020.csv') LIMIT 10;"

# Member vs Casual breakdown
duckdb -c "SELECT member_casual, COUNT(*) FROM read_csv_auto('cyclistic_tripdata_2020.csv') GROUP BY member_casual;"

# Top stations
duckdb -c "SELECT start_station_name, COUNT(*) as cnt FROM read_csv_auto('cyclistic_tripdata_2020.csv') WHERE start_station_name IS NOT NULL GROUP BY start_station_name ORDER BY cnt DESC LIMIT 10;"
```

**Using SQLite (after importing CSV):**

```bash
# Create database and import CSV
sqlite3 cyclistic.db <<EOF
.mode csv
.import cyclistic_tripdata_2020.csv trips
.headers on
.mode column
SELECT COUNT(*) as total_rows FROM trips;
SELECT member_casual, COUNT(*) FROM trips GROUP BY member_casual;
SELECT start_station_name, COUNT(*) as cnt FROM trips WHERE start_station_name IS NOT NULL GROUP BY start_station_name ORDER BY cnt DESC LIMIT 10;
EOF
```

## Alternative: Using MySQL (if you prefer)

If you want to use MySQL instead, here's how:

**On Linux:**
```bash
# Install MySQL server
sudo apt-get update
sudo apt-get install mysql-server

# Start MySQL service
sudo systemctl start mysql

# Login to MySQL
mysql -u root -p

# Create database and import CSV
CREATE DATABASE cyclistic;
USE cyclistic;
# Then use LOAD DATA INFILE or import via Python
```

**On Windows:**
- Download MySQL from https://dev.mysql.com/downloads/installer/
- Or use Docker: `docker run --name mysql-cyclistic -e MYSQL_ROOT_PASSWORD=password -d mysql`

**Note:** SQLite is simpler for this use case - no server needed!

In [4]:
import pandas as pd
datafile = r"C:\Users\imonl\OneDrive\Počítač\UCD\3rd year\2nd Trimester\COMP30770 - Programming for Big Data\cyclistic_tripdata_2020.csv"
data = pd.read_csv(datafile)

C:\Users\imonl\AppData\Local\Temp\ipykernel_11056\3156133133.py:3: DtypeWarning: Columns (0: start_station_id, 1: end_station_id) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(datafile)


In [5]:
data.info()

<class 'pandas.DataFrame'>
RangeIndex: 3541683 entries, 0 to 3541682
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             str    
 1   rideable_type       str    
 2   started_at          str    
 3   ended_at            str    
 4   start_station_name  str    
 5   start_station_id    object 
 6   end_station_name    str    
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       str    
dtypes: float64(4), object(2), str(7)
memory usage: 351.3+ MB


In [6]:
!echo "Hello world!"

"Hello world!"


In [8]:
! apt-get install mysql-server

'apt-get' is not recognized as an internal or external command,
operable program or batch file.
